In [1]:
import pandas as pd
import numpy as np
import re

from sklearn.model_selection import train_test_split

import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

import lightgbm as lgbm

/usr/lib/python3/dist-packages/scipy/__init__.py:146: UserWarning: A NumPy version >=1.17.3 and <1.25.0 is required for this version of SciPy (detected version 1.26.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"
2024-04-17 08:21:56.820602: I tensorflow/core/util/port.cc:113] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-04-17 08:21:56.821956: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-17 08:21:56.935420: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-04-17 08:21:57.230336: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical op

In [2]:
path_data = "/home/schlemuel/OneDrive/Documentos/Bancos de dados/Kaggle/automated-essay-scoring"

# Analizing

In [3]:
df_train = pd.read_csv(path_data + '/train.csv')
df_train.head()

,essay_id,full_text,score
0,000d118,Many people have car where they live. The thin...,3
1,000fe60,I am a scientist at NASA that is discussing th...,3
2,001ab80,People always wish they had the same technolog...,4
3,001bdc0,"We all heard about Venus, the planet without a...",4
4,002ba53,"Dear, State Senator\n\nThis is a letter to arg...",3


In [4]:
df_test = pd.read_csv(path_data + '/test.csv')
df_test.head()

,essay_id,full_text
0,000d118,Many people have car where they live. The thin...
1,000fe60,I am a scientist at NASA that is discussing th...
2,001ab80,People always wish they had the same technolog...


In [5]:
df_sample_submission = pd.read_csv(path_data + '/sample_submission.csv')
df_sample_submission.head()

,essay_id,score
0,000d118,3
1,000fe60,3
2,001ab80,4


In [6]:
df_train.describe()

,score
count,17307.000000
mean,2.948402
std,1.044899
min,1.000000
25%,2.000000
50%,3.000000
75%,4.000000
max,6.000000


In [7]:
maximum = 0

for line in range(len(df_train)):
    words = df_train.loc[2, 'full_text'].count(' ')

    if maximum < words:
        maximum = words

print(maximum)

555


In [8]:
df_train, df_test = train_test_split(df_train, test_size=0.12, random_state=37)

In [9]:
print(df_train.shape)
print(df_test.shape)

(15230, 3)
(2077, 3)


# Training

In [13]:
# Many

parameters = {
    'maxW': [10000, 12000],
    'lr': [0.02, 0.04],
    'num_boost_round': [700, 1000, 1500]
}

max_len = maximum + 300
quantity = 12
actual = 1

best = {'score': 0, 'parameter': {}}

for maxW in parameters['maxW']:
    for lr in parameters['lr']:
        for numBoost in parameters['num_boost_round']:
            print(f'{actual}/{quantity}')
            
            tokenizer = Tokenizer(num_words=maxW)
            tokenizer.fit_on_texts(df_train['full_text'])
            sequences_train = tokenizer.texts_to_sequences(df_train['full_text'])

            x_train = pad_sequences(sequences_train, maxlen=max_len)
            y_train = df_train['score'].values - 1
            
            # LGBM

            # Get the train and test data for the training sequence
            train_data = lgbm.Dataset(x_train, label=y_train)

            # Parameters we'll use for the prediction
            model_parameters = {
                'objective': 'multiclass',
                'metric': 'multi_logloss',
                'boosting': 'dart',
                'num_leaves': 31,
                'feature_fraction': 0.5,
                'bagging_fraction': 0.5,
                'bagging_freq': 20,
                'learning_rate': lr,
                'num_class': 6,
                'verbose': -1
            }

            # Train the classifier
            
            model = lgbm.train(model_parameters,
                                   train_data,
                                   num_boost_round=numBoost)
            
            
            sequences_test = tokenizer.texts_to_sequences(df_test['full_text'])
            x_test = pad_sequences(sequences_test, maxlen=max_len)
            
            val_pred = model.predict(x_test)
            
            submission_df = pd.DataFrame({'essay_id': [], 'score': []})

            for i in range(len(df_test)):
                submission_df = pd.concat([submission_df, pd.DataFrame([{'essay_id': df_test['essay_id'].iloc[i], 'score': np.argmax(val_pred[i]) + 1}])], ignore_index=True)
                
            correct = 0

            for i in range(len(df_test)):
                if df_test['score'].iloc[i] == submission_df['score'][i]:
                    correct += 1

            accurancy = round(correct / len(df_test), 8)
            
            if best['score'] < accurancy:
                best['score'] = accurancy
                
                best['parameter'] = {
                    'maxW': maxW,
                    'lr': lr,
                    'num_boost_round': numBoost
                }
                
            actual += 1

1/12
2/12
3/12
4/12
5/12
6/12
7/12
8/12
9/12
10/12
11/12
12/12


In [15]:
best

{'score': 0.56283101,
 'parameter': {'maxW': 10000, 'lr': 0.02, 'num_boost_round': 1000}}

In [ ]:
# {'score': 0.56283101,
# 'parameter': {'maxW': 10000, 'lr': 0.02, 'num_boost_round': 1000}}

In [61]:
parameter = {
    'maxW': 10000,
    'lr': 0.02,
    'num_boost_round': 1000
}

max_len = maximum + 300

tokenizer = Tokenizer(num_words=parameter['maxW'])
tokenizer.fit_on_texts(df_train['full_text'])
sequences_train = tokenizer.texts_to_sequences(df_train['full_text'])

x_train = pad_sequences(sequences_train, maxlen=max_len)
y_train = df_train['score'].values - 1

In [62]:
# LGBM

# Get the train and test data for the training sequence
train_data = lgbm.Dataset(x_train, label=y_train)

# Parameters we'll use for the prediction
parameters = {
    'objective': 'multiclass',
    'metric': 'multi_logloss',
    'boosting': 'dart',
    'num_leaves': 31,
    'feature_fraction': 0.5,
    'bagging_fraction': 0.5,
    'bagging_freq': 20,
    'learning_rate': parameter['lr'],
    'num_class': 6
}

# Train the classifier
model = lgbm.train(parameters,
                       train_data,
                       num_boost_round=parameter['num_boost_round'],
                       random_state=37)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.092373 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 179542
[LightGBM] [Info] Number of data points in the train set: 15230, number of used features: 855
[LightGBM] [Info] Start training from score -2.618907
[LightGBM] [Info] Start training from score -1.297752
[LightGBM] [Info] Start training from score -1.017247
[LightGBM] [Info] Start training from score -1.481421
[LightGBM] [Info] Start training from score -2.888142
[LightGBM] [Info] Start training from score -4.668178
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

...

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No f

# Predicting

In [63]:
sequences_test = tokenizer.texts_to_sequences(df_test['full_text'])
x_test = pad_sequences(sequences_test, maxlen=max_len)

In [64]:
val_pred = model.predict(x_test)

In [65]:
np.argmax(val_pred[37])

3

In [66]:
submission_df = pd.DataFrame({'essay_id': [], 'score': []})

for i in range(len(df_test)):
    submission_df = pd.concat([submission_df, pd.DataFrame([{'essay_id': df_test['essay_id'].iloc[i], 'score': np.argmax(val_pred[i]) + 1}])], ignore_index=True)

In [70]:
correct = 0

for i in range(len(df_test)):
    if df_test['score'].iloc[i] == submission_df['score'][i]:
        correct += 1
        
print(f'accurancy: {round(correct / len(df_test) * 100, 4)}%')

accurancy: 54.7906%
